In [1]:
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import time, random

In [2]:
job_links = []
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/119.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "vi-VN,vi;q=0.9,en;q=0.8",
    "Connection": "keep-alive",
}

In [3]:
def crawl_page(page):
    url = f"https://www.topcv.vn/viec-lam-it?page={page}"
    try:
        time.sleep(random.uniform(2, 4))  # delay tránh bị chặn
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            #soup = BeautifulSoup(response.content, 'html.parser')
            response.encoding = response.apparent_encoding
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            all_job = soup.find_all('div', class_='job-item-2')
            links = []
            for job in all_job:
                a_tag = job.find('a', href=True)
                if a_tag:
                    links.append(a_tag['href'])
            print(f"Trang {page}: {len(links)} links")
            return links
        else:
            print(f"Trang {page} lỗi: {response.status_code}")
    except Exception as e:
        print(f"Lỗi trang {page}: {e}")
    return []

In [12]:
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import time, random

job_links = []

# Header giả lập trình duyệt thật
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/119.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    # 👇 Tắt yêu cầu nén gzip/br
    "Accept-Encoding": "identity",
    "Accept-Language": "vi-VN,vi;q=0.9,en;q=0.8",
    "Connection": "keep-alive",
}

def crawl_page(page):
    url = f"https://www.topcv.vn/viec-lam-it?page={page}"
    try:
        # Delay ngẫu nhiên tránh bị chặn
        time.sleep(random.uniform(2, 4))

        response = requests.get(url, headers=headers, timeout=15)
        response.encoding = response.apparent_encoding  # tự đoán encoding

        # Lưu HTML của trang ra file để kiểm tra
        #with open(f"page_{page}.html", "wb") as f:
            #f.write(response.content)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            all_job = soup.find_all("div", class_="job-item-2")
            links = []
            for job in all_job:
                a_tag = job.find("a", href=True)
                if a_tag:
                    links.append(a_tag["href"])
            # print(f"Trang {page}: {len(links)} links")
            return links
        else:
            print(f"Trang {page} lỗi HTTP {response.status_code}")
    except Exception as e:
        print(f"Lỗi trang {page}: {e}")
    return []


# === Chạy đa luồng nhẹ (đừng quá cao nếu không bị chặn) ===
with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    results = executor.map(crawl_page, range(2, 22))  # thử trước 20 trang
    for r in results:
        job_links.extend(r)

print("\nTổng số link thu được:", len(job_links))



Tổng số link thu được: 1000


In [29]:
job_links[0]

'https://www.topcv.vn/brand/fptis/tuyen-dung/nhan-vien-kiem-thu-tester-j1903857.html?ta_source=ITJobs_LinkDetail'

In [30]:
url = "https://www.topcv.vn/viec-lam/java-developer/1907639.html?ta_source=ITJobs_LinkDetail"
# url = job_links[0]
#Hai link trên nó khác nhau một link của trang premium, một trang của công ty thường
#Đoạn code xử lý phía dưới đều crawl được
response = requests.get(url, headers=headers, timeout=15)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    # print(soup)

In [37]:
title_tag = soup.find(lambda tag: tag.name in ["h2", "h3", "strong"] and "Mô tả công việc" in tag.get_text())
job_description_content = soup.find("div", class_= lambda c: c and "job-description" in c) #Mekeep, sai dấu gạch dưới và gạch giữa mà tìm mãi.
#print(title_tag) #Đã lấy được thẻ title_tag
print(job_description_content) #Đã lấy được job_description


<div class="job-description">
<div class="job-description__item">
<h3>Mô tả công việc</h3>
<div class="job-description__item--content">
<ul><li>Chịu trách nhiệm về việc thiết kế, viết mã nguồn và triển khai các ứng dụng Java dựa trên các yêu cầu kỹ thuật và thiết kế đã được phê duyệt.</li><li>Viết mã nguồn sạch, tối ưu hóa và dễ bảo trì; thực hiện code review để đảm bảo mã nguồn tuân theo các nguyên tắc lập trình tốt nhất và các chuẩn của công ty.</li><li>Làm việc chặt chẽ với nhóm để xác định các cơ hội tối ưu hóa mã nguồn và cải thiện hiệu suất của ứng dụng.</li><li>Phản hồi nhanh chóng và hiệu quả đối về các vấn đề kỹ thuật phát sinh trong quá trình phát triển và sau khi sản phẩm được triển khai.</li><li>Phối hợp với các nhóm khác như UI/UX, QA, và DevOps để đảm bảo ứng dụng được phát triển một cách toàn diện.</li><li>Luôn cập nhật các công nghệ mới nhất và áp dụng vào trong công việc hiệu quả.</li></ul>
</div>
</div>
<div class="job-description__item">
<h3>Yêu cầu ứng viên</h3>
<di

In [44]:
mota = []
if job_description_content:
    #Lấy thẻ tiếp theo ngay sau nó, thường là div hoặc ul
    mota = job_description_content.find_all("li")
    print(mota)
    print(type(mota))

[<li>Chịu trách nhiệm về việc thiết kế, viết mã nguồn và triển khai các ứng dụng Java dựa trên các yêu cầu kỹ thuật và thiết kế đã được phê duyệt.</li>, <li>Viết mã nguồn sạch, tối ưu hóa và dễ bảo trì; thực hiện code review để đảm bảo mã nguồn tuân theo các nguyên tắc lập trình tốt nhất và các chuẩn của công ty.</li>, <li>Làm việc chặt chẽ với nhóm để xác định các cơ hội tối ưu hóa mã nguồn và cải thiện hiệu suất của ứng dụng.</li>, <li>Phản hồi nhanh chóng và hiệu quả đối về các vấn đề kỹ thuật phát sinh trong quá trình phát triển và sau khi sản phẩm được triển khai.</li>, <li>Phối hợp với các nhóm khác như UI/UX, QA, và DevOps để đảm bảo ứng dụng được phát triển một cách toàn diện.</li>, <li>Luôn cập nhật các công nghệ mới nhất và áp dụng vào trong công việc hiệu quả.</li>, <li>Kiến thức chuyên sâu về Java với ít nhất 02 năm kinh nghiệm làm việc.</li>, <li>Kinh nghiệm làm việc với các framework như Spring, Hibernate và Quarkus, và khả năng viết các services RESTful và SOAP.</li>, <l